In [20]:
import pandas as pd
import glob
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [34]:
# Grayson's path
#train_folder = "C:\\Users\\grays\\Will-Grayson GitHub Repo\\will-grayson-ML\\nineteen-class\\data\\train\\"
#test_folder = "C:\\Users\\grays\\Will-Grayson GitHub Repo\\will-grayson-ML\\nineteen-class\\data\\test\\"

# Will's path (comment out if not in use)
train_folder = "C:\\Users\\willg\\OneDrive\\CSCI\\summer-2024-work\\will-grayson-ML\\nineteen-class\\data\\train\\"
test_folder = "C:\\Users\\willg\\OneDrive\\CSCI\\summer-2024-work\\will-grayson-ML\\nineteen-class\\data\\test\\"

# Function to load and concatenate CSV files from a list of file paths
def load_and_concat(files, sample_fraction=None):
    df_list = []
    for file in files:
        df = pd.read_csv(file)
        if sample_fraction:
            df = df.sample(frac=sample_fraction, random_state=42)  # Random sampling
        df_list.append(df)
    return pd.concat(df_list, ignore_index=True)

# List all CSV files in the train and test folders
train_files = glob.glob(train_folder + "*.csv")
test_files = glob.glob(test_folder + "*.csv")

# Load and concatenate training and testing data
train_df = load_and_concat(train_files, sample_fraction=1)  # Use 10% of the data
test_df = load_and_concat(test_files, sample_fraction=1)


In [22]:
# Combine target labels into a single column for multi-class classification
# Assuming the labels are one-hot encoded
target_columns = ['spoofing', 'benign', 'MQTT_DDoS_Connect_Flood', 'MQTT_DoS_Connect_Flood', 
    'MQTT_DDoS_Publish_Flood', 'MQTT_DoS_Publish_Flood', 'MQTT_Malformed_Data',
    'Recon_OS_Scan', 'Recon_Ping_Sweep', 'Recon_Port_Scan', 'Recon_VulScan',
    'DoS_ICMP', 'DoS_SYN', 'DoS_TCP', 'DoS_UDP',
    'DDoS_ICMP', 'DDoS_SYN', 'DDoS_TCP', 'DDoS_UDP'
]
feature_columns = [col for col in train_df.columns if col not in target_columns]

# Separate features and labels
X_train = train_df[feature_columns]
y_train = train_df[target_columns]
X_test = test_df[feature_columns]
y_test = test_df[target_columns]


In [23]:
# Encode the labels to numerical values
label_encoder = LabelEncoder()
y_train_labels = label_encoder.fit_transform(y_train.idxmax(axis=1))
y_test_labels = label_encoder.transform(y_test.idxmax(axis=1))

# Create a scaler object
scaler = MinMaxScaler()

# Fit the scaler to the training features and transform both training and testing features
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [35]:
# Initialize the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=20, n_jobs=-1)

# Train the model
model.fit(X_train_scaled, y_train_labels)

# Make predictions
y_pred_labels = model.predict(X_test_scaled)

# Evaluate the model
print(confusion_matrix(y_test_labels, y_pred_labels))
print(classification_report(y_test_labels, y_pred_labels, target_names=label_encoder.classes_))

[[34957     0     0    13     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0]
 [    1 17198     6    27     1     2     0     1     0     1     0     0
      0     0     0     1     0     2     0]
 [    0     0 18253     3     0     2     2     0     0     0     0     0
      0     0     0     0     0     0     0]
 [    6     1     6 36187     0     1     0     6     0     0     0     0
      0     0     0     0     0     0     0]
 [    5     0     0     0  9829     0     2     7     0     0     0     0
      0     0     0     0     0     0     0]
 [    0     8     0     0     0  9852     0     0     0     0     0     0
      0     0     0     0     0     0     0]
 [    0     0     1     0     0     0  8209     0     0     0     0     0
      0     0     0     0     0     0     0]
 [    0     0     1     6     1     1     1 13745     0     0     0     0
      0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     1     

In [32]:
results = pd.DataFrame({
    'actual': label_encoder.inverse_transform(y_test_labels),
    'predicted': label_encoder.inverse_transform(y_pred_labels)
})

# Print results where actual value is 'MQTT_DDoS_Publish_Flood'
print(results[results['actual'] == 'Recon_VulScan'])

              actual        predicted
12971  Recon_VulScan    Recon_OS_Scan
12972  Recon_VulScan    Recon_VulScan
12973  Recon_VulScan           benign
12974  Recon_VulScan           benign
12975  Recon_VulScan           benign
12976  Recon_VulScan    Recon_VulScan
12977  Recon_VulScan           benign
12978  Recon_VulScan           benign
12979  Recon_VulScan           benign
12980  Recon_VulScan           benign
12981  Recon_VulScan    Recon_VulScan
12982  Recon_VulScan    Recon_VulScan
12983  Recon_VulScan    Recon_VulScan
12984  Recon_VulScan    Recon_OS_Scan
12985  Recon_VulScan           benign
12986  Recon_VulScan           benign
12987  Recon_VulScan           benign
12988  Recon_VulScan           benign
12989  Recon_VulScan           benign
12990  Recon_VulScan    Recon_VulScan
12991  Recon_VulScan    Recon_OS_Scan
12992  Recon_VulScan    Recon_VulScan
12993  Recon_VulScan           benign
12994  Recon_VulScan    Recon_VulScan
12995  Recon_VulScan           benign
12996  Recon